In [ ]:
from orion import Orion
import pandas as pd
import shutil
import os
import csv
from orion.data import load_signal, load_anomalies

In [ ]:
#specify which dataset you want to train on
dataset = "CWRU"

In [3]:
#specify train and test task
train_task = 0
test_task = 0
sample_freq = "12k"
fault_location = "DE"

In [4]:
#specify sensor
sensor = "DE"

In [5]:
#set path where all folders to run code are
base_path = "/home/lunet/cosoc/Desktop/orion_anomaly_pipeline"


In [6]:
#set experiment name
experiment = "CWRU_test"

In [7]:
#gets the data
def get_data(mode, fault_type=""):
    if mode == "train":
        task = "normal_" + str(train_task)
        path = os.path.join(base_path, "datasets", dataset, mode, task, sensor) + ".csv"
        data = pd.read_csv(path)
    
    elif mode == "test":
        if fault_type == "normal":
            task = "normal_" + str(train_task)
            path = os.path.join(base_path, "datasets", dataset, mode, task, sensor) + ".csv"
            data = pd.read_csv(path)
        else:
            fault = fault_type + "_" + test_task
            path = os.path.join(base_path, "datasets", dataset, mode, sample_freq, fault_location, fault, sensor) + ".csv"
            data = pd.read_csv(path)
    
    return data
            
            
            
            
            

In [8]:
#set the hyperparameters for training
hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.time_segments_aggregate#1": {
        "time_column": "timestamp",
        "interval": 10,
        "method": "mean"
    },
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences#1": {
         "window_size": 100
     },
    "orion.primitives.tadgan.TadGAN#1": {
        "epochs": 50,
        "batch_size": 64
    },
    "orion.primitives.tadgan.score_anomalies#1": {
        "rec_error_type": "dtw",
        "comb": "mult"
    }
}

In [9]:
#gets the train data
train_data = get_data(mode="train")
train_data

,timestamp,value
0,0,0.053197
1,1,0.088662
2,2,0.099718
3,3,0.058621
4,4,-0.004590
...,...,...
146358,146358,0.052362
146359,146359,0.050276
146360,146360,0.076770
146361,146361,0.110983


In [10]:
model = Orion(pipeline='tadgan',hyperparameters=hyperparameters)
model.fit(train_data)

2023-05-24 14:07:33.193459: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-05-24 14:07:38.862457: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-05-24 14:07:38.884929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 14:07:38.885207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2023-05-24 14:07:38.885224: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-05-24 14:07:38.886162: I tensorflow/stream_execut

2023-05-24 14:07:46.377248: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-05-24 14:07:46.515325: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


Epoch: 1/50, Losses: {'cx_loss': -3.1772, 'cz_loss': -4.6185, 'eg_loss': 6.5}
Epoch: 2/50, Losses: {'cx_loss': -6.8175, 'cz_loss': 1.1689, 'eg_loss': -54.3469}
Epoch: 3/50, Losses: {'cx_loss': -1.7364, 'cz_loss': 1.8828, 'eg_loss': -60.127}
Epoch: 4/50, Losses: {'cx_loss': -0.6263, 'cz_loss': -0.509, 'eg_loss': -36.0898}
Epoch: 5/50, Losses: {'cx_loss': -0.6647, 'cz_loss': 1.2202, 'eg_loss': -37.2646}
Epoch: 6/50, Losses: {'cx_loss': -0.9847, 'cz_loss': 0.1252, 'eg_loss': -31.8388}
Epoch: 7/50, Losses: {'cx_loss': -0.9175, 'cz_loss': 0.3068, 'eg_loss': -36.0029}
Epoch: 8/50, Losses: {'cx_loss': -0.9217, 'cz_loss': 0.6784, 'eg_loss': -37.4567}
Epoch: 9/50, Losses: {'cx_loss': -0.9712, 'cz_loss': 1.3813, 'eg_loss': -30.1877}
Epoch: 10/50, Losses: {'cx_loss': -1.3612, 'cz_loss': -0.9499, 'eg_loss': -33.7814}
Epoch: 11/50, Losses: {'cx_loss': -1.2576, 'cz_loss': 2.5095, 'eg_loss': -42.2606}
Epoch: 12/50, Losses: {'cx_loss': -0.4273, 'cz_loss': 2.5992, 'eg_loss': -23.6395}
Epoch: 13/50, Los

In [11]:
file_labels = pd.read_csv(os.path.join(base_path, "datasets_unprocessed", dataset, "labels.csv"))

In [12]:
unfiltered_files = file_labels.loc[(file_labels["motor_load"] == test_task) & (file_labels["sample_freq"] == int(sample_freq[0:2]))]                                                  
test_files = unfiltered_files.loc[(unfiltered_files["fault_location"] == fault_location) | (unfiltered_files["fault"] == "normal")]

In [13]:
test_files

,file_name,label,fault_location,fault_diameter,motor_load,rpm,sample_freq,fault
0,97,normal_0,NaN,NaN,0,1797,12,normal
4,105,IR007_0,DE,0.007,0,1797,12,IR
8,169,IR014_0,DE,0.014,0,1797,12,IR
12,209,IR021_0,DE,0.021,0,1797,12,IR
16,3001,IR028_0,DE,0.028,0,1797,12,IR
20,118,B007_0,DE,0.007,0,1797,12,B
24,185,B014_0,DE,0.014,0,1797,12,B
28,222,B021_0,DE,0.021,0,1797,12,B
32,3005,B028_0,DE,0.028,0,1797,12,B
36,130,OR007@6_0,DE,0.007,0,1797,12,OR


In [20]:
import orion.evaluation
test_data = pd.read_csv("/home/lunet/cosoc/Desktop/orion_anomaly_pipeline/datasets/CWRU/test/normal_0/DE.csv")
test_data

,timestamp,value
0,146363,0.056118
1,146364,0.047564
2,146365,0.065922
3,146366,0.100970
4,146367,0.094503
...,...,...
97570,243933,-0.059664
97571,243934,-0.063836
97572,243935,-0.034630
97573,243936,0.016689


In [15]:
detected_anomalies = model.detect(test_data)

In [16]:
detected_anomalies

,start,end,severity
0,56410,57500,0.094772
1,83310,84890,0.240467
2,111750,113100,0.310292


In [ ]:
b = list(range(0,len(test_data), 10000))
b

In [ ]:
for i in list(range(0,len(test_data), 1000))[0:-1]:
    start = i
    end = i + 1000
    sequence = test_data[start:end]
    
    

In [ ]:
import orion.evaluation
start = []
end = []
severity = []
key = []
n = 0

    


In [21]:

for i in list(range(0,len(test_data), 10000))[0:-1]:
    start = i
    end = i + 10000
    sequence = test_data[start:end] 
    detected_anomalies = model.detect(sequence)
    print(detected_anomalies)

    start     end  severity
0  150533  151593  0.281156
    start     end  severity
0  161213  162253  0.086102
    start     end  severity
0  166363  167093  0.051041
    start     end  severity
0  182433  183463  0.105880
1  184063  185113  0.265870
2  185643  186343  0.066751
    start     end  severity
0  190473  190973  0.097064
    start     end  severity
0  196953  197973  0.039626
1  199623  200663  0.092220
2  202943  203983  0.226080
    start     end  severity
0  209223  210263  0.142481
1  212303  213463  0.077804
    start     end  severity
0  217783  218813  0.210077
1  222223  223273  0.124576
2  223753  224603  0.258116
    start     end  severity
0  229013  230053  0.131380
1  232213  233233  0.063281
2  234903  235933  0.083730


In [ ]:
start = []
end = []
severity = []
key = []
start.append(2)
start.append(3)

In [ ]:
for index, row in detected_anomalies.iterrows():
    start.append(row["start"])
    end.append(row["end"])
    severity.append(row["severity"])
    key = n

In [ ]:
dict = {"start": [], "end": [], "severity": [], "key": []}
df = pd.DataFrame(dict)
new_df = df.append(df2)
new_df

In [ ]:
for index, row in test_files.iterrows():
    test_data = get_data(mode="test", fault_type=row["fault"])

In [ ]:
detected_anomalies = model.detect(test_data)